In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install transformers
!pip install sentencepiece
!pip install tf_sentencepiece
!pip install rouge-score

     |████████████████████████████████| 1.5MB 7.2MB/s 
     |████████████████████████████████| 890kB 34.7MB/s 
     |████████████████████████████████| 2.9MB 35.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c01ae317d15dff774c5386fab063c8dbf26d178217d0ed84da2782f8fba38f68
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.1MB 6.8MB/s 
     |████████████████████████████████| 2.1MB 6.5MB/s 


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import torch
import sentencepiece as spm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, TrainingArguments, Trainer
from transformers import BartForConditionalGeneration, BartTokenizer
import tqdm
from time import sleep

In [4]:
(ds_train, ds_validation), ds_info = tfds.load(
    'cnn_dailymail',
    split = ['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cnn_dailymail/plain_text/3.0.0
INFO:absl:Load dataset info from /tmp/tmp4ii84py9tfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset cnn_dailymail (/root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)


INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfTHk4NFg2SndKG8BdJPpt2iRo6Dpzz23CByJuAePEilB-pxbcBCHaWDs.tmp.55bf18b183b941f4b69131b8ea329a3c...
INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfM1BxdkxVaTY2zVV-G71RIXPssrrvSAjt19Cy91r-9CQ2F9DMKA0uFk0.tmp.464c224676da4651874e64a162cc9212...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_test.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url_list_a705_isK790OHOPsIZX-ACsObq_vchU9r5Uduh6ULX3c.txt.tmp.6098ee30a8864b91bb445f139c5834e1...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url

INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-train.tfrecord. Shard lengths: [17945, 17944, 17945, 17944, 17945, 17944, 17945, 17944, 17945, 17945, 17944, 17945, 17944, 17945, 17944, 17945]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-validation.tfrecord. Shard lengths: [13368]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteK6RQSS/cnn_dailymail-test.tfrecord. Shard lengths: [11490]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split ['train', 'validation'], from /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [5]:
ds_info

tfds.core.DatasetInfo(
    name='cnn_dailymail',
    version=3.0.0,
    description='CNN/DailyMail non-anonymized summarization dataset.

There are two features:
  - article: text of news article, used as the document to be summarized
  - highlights: joined text of highlights with <s> and </s> around each
    highlight, which is the target summary',
    homepage='https://github.com/abisee/cnn-dailymail',
    features=FeaturesDict({
        'article': Text(shape=(), dtype=tf.string),
        'highlights': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=311971,
    splits={
        'test': 11490,
        'train': 287113,
        'validation': 13368,
    },
    supervised_keys=('article', 'highlights'),
    citation="""@article{DBLP:journals/corr/SeeLM17,
      author    = {Abigail See and
                   Peter J. Liu and
                   Christopher D. Manning},
      title     = {Get To The Point: Summarization with Pointer-Generator Networks},
      journal   = {Co

In [6]:
# model_name = 'facebook/bart-base'
# local_model_path = '/content/drive/MyDrive/NLP Group Project/Models/bart_cnn_model/checkpoint-20000'
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = BartForConditionalGeneration.from_pretrained(local_model_path).to(torch_device)
# tokenizer = BartTokenizer.from_pretrained(model_name)

In [6]:
# model_name = 'google/pegasus-xsum'
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Load model
model_name = 'facebook/bart-base'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [7]:
# Freeze Embedding and Encoder Layers
for name, param in model.base_model.named_parameters():
  if name.split('.') == 'encoder':
    param.requires_grad = False
  else:
    param.requires_grad = True

In [8]:
def strings_from_tensors(tensors):
  texts = [text.numpy().decode() for text, label in tensors]
  labels = [label.numpy().decode() for text, label in tensors]
  return texts, labels

In [10]:
# Generate Strings of Text and Labels for BART tokenization
train_texts, train_labels = strings_from_tensors(ds_train)
val_texts, val_labels = strings_from_tensors(ds_validation)

In [11]:
# val_summs = []
# for text in val_texts:  
#   batch = tokenizer.prepare_seq2seq_batch(src_texts=[text], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
#   summarized = model.generate(**batch)
#   summ_text = tokenizer.batch_decode(summarized, skip_special_tokens=True)
#   val_summs += summ_text


In [11]:
# tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True)
print('Encodings Generated')
train_labels = tokenizer(train_labels, truncation=True, padding=True)
val_labels = tokenizer(val_labels, truncation=True, padding=True)
# test_labels = tokenizer(test_labels, truncation=True, padding=True)
print('Labels Generated')


Encodings Generated
Labels Generated


In [12]:
class SummaryData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, size):
        self.encodings = encodings
        self.labels = labels
        self.size = size

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return self.size

In [13]:
# Generate predictions using test for CNN/DailyMail Validation set
train_data = SummaryData(train_encodings, train_labels, len(train_texts))
val_data = SummaryData(val_encodings, val_labels, len(val_texts))


In [ ]:
# Train using built in Hugging Face Train Objects

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP Group Project/Models/checkpoints/bart_cnn_model',
    overwrite_output_dir = True,           
    num_train_epochs=10,                       # total number of training epochs
    per_device_train_batch_size=2,                # batch size per device during training
    per_device_eval_batch_size=2,
    # evaluation_strategy = 'epoch'                 # batch size for evaluation
    warmup_steps=2000,                               
    learning_rate=1e-5,                            # strength of weight decay
    logging_dir='./logs',                         # directory for storing logs
    logging_steps=4000,
    save_steps = 8000,
    save_total_limit = 3

)

trainer = Trainer(
    model=model,                                  # the instantiated model to be trained
    args=training_args,                           # training arguments, defined above
    train_dataset=train_data,                     # training dataset
    eval_dataset=val_data,
)

trainer.train()
model.save_pretrained('/content/drive/MyDrive/NLP Group Project/Models/bart_cnn_model')




Step,Training Loss
4000,1.807365
8000,0.260304
12000,0.253349
16000,0.251331
20000,0.250697
24000,0.251206
28000,0.246232
32000,0.246257
36000,0.245875
40000,0.245698


In [ ]:
# Now let's pull in evaluation data
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir("/content/drive/My Drive/NLP Group Project/")
# !pwd
# !ls
# model.train()

# optim = AdamW(model.parameters(), lr=5e-5)
# train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

# for epoch in range(3):
#   with tqdm.tqdm(train_loader, unit="batch") as train_epoch:
#     for batch in train_epoch:
#       train_epoch.set_description(f"Epoch {epoch}")
#       optim.zero_grad()
#       input_ids = batch['input_ids'].to(torch_device)
#       attention_mask = batch['attention_mask'].to(torch_device)
#       labels = batch['labels'].to(torch_device)
#       outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs[0]
#       loss.backward()
#       optim.step()
#       sleep(0.1)

# model.eval()

In [ ]:
# import pandas as pd
# bbc_news = pd.read_csv('bbc_news_and_summary.csv')
# bbc_news = bbc_news[['Article with stop words', 'Summary']]
# bbc_news = bbc_news.rename(columns={'Article with stop words':'Article'})
# bbc_articles = bbc_news.Article.to_list()


In [ ]:
val_predictions = []
val_sample = val_texts[:99]
for text in val_sample:
  batch = tokenizer.prepare_seq2seq_batch(src_texts=[text], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
  summarized = model.generate(**batch)
  summ_text = tokenizer.batch_decode(summarized, skip_special_tokens=True)
  val_predictions += summ_text




In [ ]:
from rouge import Rouge 
import nltk.translate.bleu_score as bleu

# ROUGE-N is an n-gram recall between a candidate summary and a set of reference summaries
# BLEU - the closer a machine translation is to a professional human translation, the better it is
# a summary-level evaluation

def eval(reference_summary,generated_summary):

  rouge = Rouge()
  rouge_scores = rouge.get_scores(generated_summary, reference_summary)
  for score in scores:
    r1_score = score['rouge-1']['r']
    r2_score = score['rouge-2']['r']

  ref = reference_summary.split()
  hyp = generated_summary.split()
  bleu_score = bleu.sentence_bleu(ref,hyp)
  # smoothing
  # smoothie = bleu.SmoothingFunction()
  # smo_bleu_score = bleu.sentence_bleu(ref,hyp, smoothing_function=smoothie.method1)
  
  return r1_score,r2_score,bleu_score

In [ ]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"
reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students ' knowledge of even ts in the news"

r1_score,r2_score,bleu_score = eval(hypothesis,reference)

print('ROUGE-1 score is:', r1_score)
print('ROUGE-2 score is:', r2_score)
print('BLEU score is:', bleu_score)
# print('Smoothing BLEU score is:', smo_bleu_score)